In [ ]:
import pandas as pd
import numpy as np
import os
#from imblearn.combine import SMOTEENN
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn import linear_model
from sklearn.metrics import classification_report
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, MinMaxScaler ,normalize
from sklearn import preprocessing
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
import seaborn as sns
from collections import Counter
%matplotlib inline
mpl.style.use( 'ggplot' )
sns.set_style( 'white' )
pylab.rcParams[ 'figure.figsize' ] = 8 , 6


In [ ]:
df = pd.read_csv('../../data/Data_set.csv',encoding='cp949')
df1 = pd.read_csv('../../data/Test_set.csv',encoding='cp949')
df1.AGE.value_counts()

In [ ]:
def modefinder(numbers):
    c = Counter(numbers)
    mode = c.most_common(1)
    return mode[0][0]

In [ ]:
def age_mean(columns):
    for i in range(len(columns)):
        if (columns[i] == '*') == True:
            del columns[i]
    return np.mean(np.array(columns),dtype=np.float64 )

In [ ]:
def mode_paym_metd(columns): #for sex, paym_metd
    null_column = df[columns.isnull() == True].index.tolist()
    for i in null_column:
            columns[i] = modefinder(columns)
    return columns

In [ ]:
def mode_sex(columns): #for sex, paym_metd
    for i in range(len(columns)):
        if (columns[i] == '*') == True:
            columns[i] = modefinder(columns)
    return columns

In [ ]:
df.SEX= mode_sex(df.SEX)
df.PAYM_METD = mode_paym_metd(df.PAYM_METD)
mean_age = age_mean(df.AGE)
df.AGE = age_finder(df.AGE)
df.LAST_CHLD_AGE = chld_age_fonder(df.LAST_CHLD_AGE)

In [ ]:
def age_finder(columns):
    j = 15
    k = -1
    while j < 70:
        j = j+5
        k = k+5
        columns = np.where((columns == "*") & (k < df["LAST_CHLD_AGE"]) & (df["LAST_CHLD_AGE"] <= k+5), j+5 ,columns)
        columns = np.where((columns == "*") & (k < df["LAST_CHLD_AGE"]) & (df["LAST_CHLD_AGE"] ==0), mean_age ,columns) 
        columns = np.where((columns == "*"),mean_age,columns)
        if j >70:
            break
    return columns

In [ ]:
def chld_age_finder(age_column,child_column):
    i = 15
    l = 4
    child_age = []
    non_zero_list = []
    while i <70:
        i = i+5
        l = l+5
        del child_age[:]
        del non_zero_list[:]
        age_index = df[age_column == i].index.tolist()
        for j in age_index:
            child_age.append(df["LAST_CHLD_AGE"][j])
        child_age = pd.DataFrame(child_age)
        child_age = child_age.dropna()
        for k in range(len(child_age)):
            if child_age[k] > i - 10:
                child_age[k] = mode_finder(child_age)
        for non_zero in child_age:
            if non_zero != 0:
                non_zero_list.append(non_zero)
        mean_age_child = np.mean(non_zero_list)
        child_column = np.where((df[child_column.isnull() == True]) & (df["AGE"] == 70),mean_age_child,child_column)
        child_column = np.where((df[child_column.isnull() == True]) & (i <= df["AGE"])&(df["AGE"] < i+5),mean_age_child,child_column)
        child_column = np.where((df[child_column.isnull() == True]),modefinder(df.LAST_CHLD_AGE),child_column)
        if  i > 70:
            break

    return child_column